In [1]:
import sqlite3
from sqlite3 import Error
import os
import bcrypt

# Criar tabelas

In [2]:
#CRIA A CONEXÃO COM O BANCO DE DADOS
def create_connection(db_file):
    """ Cria conexão com o banco de dados SQLite
        especificado por arquivo_db
        :param arquivo_db: diretório do banco de dados
        :return : Objeto de conexão ou None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        return e

    return conn

In [3]:
# CRIA UMA FUNÇÃO GERADORA DE TABELAS
def create_table(conn, create_table_sql):

    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        return e

In [4]:
def main():
    database = "fluxo.db"

    sql_create_movimentacoes_table = """CREATE TABLE IF NOT EXISTS movimentacoes(
                                    id_movimentacao INTEGER primary key autoincrement,
                                    data DATE not null,
                                    tipo TEXT(50) not null,
                                    id_categoria INTEGER not null,
                                    id_conta INTEGER not null,
                                    comentario TEXT(300),
                                    valor REAL not null
                                ); """
    

    sql_create_conta_table = """CREATE TABLE IF NOT EXISTS conta(
                                id_conta INTEGER primary key autoincrement,
                                nome TEXT(100) not null,
                                saldo REAL
                                ); """
    
    sql_create_categoria = """CREATE TABLE IF NOT EXISTS categoria(
                                id_categoria INTEGER primary key autoincrement,
                                tipo TEXT(50) not null,
                                nome TEXT(50) not null
                                )"""
    sql_create_password = """CREATE TABLE IF NOT EXISTS password(
                                password 
                                )"""
    
    # Cria a conexão com o banco de dados
    conn = create_connection(database)

    # Cria as tabelas
    if conn is not None:
        
        # cria a tabela movimentações
        create_table(conn, sql_create_movimentacoes_table)

        # cria a tabela conta
        create_table(conn, sql_create_conta_table)
        
        # cria a tabela categoria
        create_table(conn, sql_create_categoria)
        
        # cria a tabela password
        create_table(conn, sql_create_password)
        
        
    else:
        print("Error! cannot create the database connection.")



In [6]:
main()

# Insert

In [6]:
def adicionar_conta(nome : str):
    database = "fluxo.db"
    conn = create_connection(database)
    sql = ''' INSERT INTO conta(nome)
              VALUES(?)'''
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, (nome,))
            conn.commit()
            return cur.lastrowid
        
        except Error as e:
            return e

In [8]:
adicionar_conta("PicPay")

2

In [9]:
def adicionar_senha(senha : str):
    database = "fluxo.db"
    conn = create_connection(database)
    sql = ''' INSERT INTO password(password)
              VALUES(?)'''
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, (senha,))
            conn.commit()
            return cur.lastrowid
        
        except Error as e:
            return e

In [25]:
def adicionar_categoria(tipo : str, nome : str):
    database = "fluxo.db"
    conn = create_connection(database)
    sql = ''' INSERT INTO categoria(tipo, nome)
              VALUES(?,?)'''
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, (tipo, nome))
            conn.commit()
            return cur.lastrowid
        
        except Error as e:
            return e

In [28]:
adicionar_categoria("Entrada", "Vendas")

4

# SELECT 

In [12]:
# Função para que as consultas SQL retornem uma lista de dicionários ao invés de uma lista de tuplas
def dict_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    return {key: value for key, value in zip(fields, row)}


In [18]:
def consultar_senha():
    database = "fluxo.db"
    conn = create_connection(database)
    conn.row_factory = dict_factory
    
    sql = "SELECT * FROM password"
    
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql)
            consulta = cur.fetchone()
            
            return consulta
            
        except Error as e:
            return e
consultar_senha()

{'password': 7593258303979883720}

In [15]:
def consultar_contas():
    database = "fluxo.db"
    conn = create_connection(database)
    conn.row_factory = dict_factory
    
    sql = "SELECT * FROM conta"
    
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql)
            consulta = cur.fetchall()
            
            return consulta
            
        except Error as e:
            return e

In [18]:
contas = consultar_contas()
print(contas)

[{'id_conta': 1, 'nome': 'Neon', 'saldo': None}, {'id_conta': 2, 'nome': 'PicPay', 'saldo': None}]


In [20]:
contas_lista = []
for i in contas:
    contas_lista.append(i['nome'])
print(contas_lista)

['Neon', 'PicPay']


In [23]:
def consultar_categorias(tipo : str):
    database = "fluxo.db"
    conn = create_connection(database)
    conn.row_factory = dict_factory
    
    sql = "SELECT * FROM categoria WHERE tipo = ?"
    
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, (tipo,))
            consulta = cur.fetchall()
            
            return consulta
            
        except Error as e:
            return e

In [31]:
categorias = consultar_categorias("Entrada")
cat_lista = []
for i in categorias:
    cat_lista.append(i['nome'])
print(cat_lista)

['Aulas', 'Salário', 'Empréstimo', 'Vendas']


In [1]:
import db_manager as dbm
dbm.id_conta("Neon")

1

In [2]:
import db_manager as dbm
dbm.id_categoria("Vendas")

4

In [4]:
import db_manager as dbm
import pandas as pd
base = dbm.consulta_base()
df = pd.DataFrame(base)
for i in base:
    print(i)
df

{'id_movimentacao': 1, 'data': '2024-05-21', 'tipo': 'Entrada', 'id_categoria': 1, 'id_conta': 2, 'comentario': 'Alice', 'valor': 250.0, 'conta': 'Neon', 'saldo': None, 'categoria': 'Aulas'}
{'id_movimentacao': 2, 'data': '2024-05-21', 'tipo': 'Entrada', 'id_categoria': 1, 'id_conta': 2, 'comentario': 'Rafael', 'valor': 250.0, 'conta': 'Neon', 'saldo': None, 'categoria': 'Aulas'}
{'id_movimentacao': 3, 'data': '2024-05-21', 'tipo': 'Entrada', 'id_categoria': 2, 'id_conta': 3, 'comentario': 'Website SAS', 'valor': 750.0, 'conta': 'PicPay', 'saldo': None, 'categoria': 'Vendas'}
{'id_movimentacao': 4, 'data': '2024-05-21', 'tipo': 'Saída', 'id_categoria': 4, 'id_conta': 1, 'comentario': 'Aluguel Maio', 'valor': 500.0, 'conta': 'Banco do Brasil', 'saldo': None, 'categoria': 'Aluguel'}
{'id_movimentacao': 5, 'data': '2024-05-21', 'tipo': 'Saída', 'id_categoria': 5, 'id_conta': 2, 'comentario': 'Pizza', 'valor': 70.0, 'conta': 'Neon', 'saldo': None, 'categoria': 'Alimentação'}


,id_movimentacao,data,tipo,id_categoria,id_conta,comentario,valor,conta,saldo,categoria
0,1,2024-05-21,Entrada,1,2,Alice,250.0,Neon,None,Aulas
1,2,2024-05-21,Entrada,1,2,Rafael,250.0,Neon,None,Aulas
2,3,2024-05-21,Entrada,2,3,Website SAS,750.0,PicPay,None,Vendas
3,4,2024-05-21,Saída,4,1,Aluguel Maio,500.0,Banco do Brasil,None,Aluguel
4,5,2024-05-21,Saída,5,2,Pizza,70.0,Neon,None,Alimentação


# Testes 

In [12]:
os.path.exists("fluxo.db")

True

In [24]:
word = "johnny*03"
print(hash(word))

senha = hash(word)
adicionar_senha(senha)

result = consultar_senha()

if result['password'] == senha:
    print(True)
else:
    print(result)

7593258303979883720
True


In [2]:
senha = b"johnny*03"
hashed = bcrypt.hashpw(senha, bcrypt.gensalt())

In [4]:
type(str(hashed))

str

In [5]:
print(hashed)

b'$2b$12$bEPTbPYRv94uicpoVynDwuljfdt76RSflS8hJNY/pAz6awF8l72G.'


In [6]:
type(hashed)

bytes

In [7]:
pip install pwinput

  Using cached pwinput-1.0.3.tar.gz (4.4 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pwinput: filename=pwinput-1.0.3-py3-none-any.whl size=4531 sha256=770284ba6054945ea57524f4a0005bfed2fdd476eb56fff3748ea1090fa659f1
  Stored in directory: c:\users\johnny\appdata\local\pip\cache\wheels\84\4f\86\62eb6202eb45b529f3dde2b93ba7eda76e7b782f4256f15c8d
Successfully built pwinput
Note: you may need to restart the kernel to use updated packages.


In [1]:
import db_manager as dbm
import bcrypt
import pwinput

result = dbm.consultar_senha()
"""
while True:
    senha = pwinput.pwinput(prompt='Insira a senha para acessar o sistema: ')
    if bcrypt.checkpw(bytes(senha['password'], encoding='utf-8'), result):
        subprocess.run(["streamlit", "run", "Home.py"])
    else:
        print("Senha incorreta. Tente novamente")
            
"""

'\nwhile True:\n    senha = pwinput.pwinput(prompt=\'Insira a senha para acessar o sistema: \')\n    if bcrypt.checkpw(bytes(senha[\'password\'], encoding=\'utf-8\'), result):\n        subprocess.run(["streamlit", "run", "Home.py"])\n    else:\n        print("Senha incorreta. Tente novamente")\n            \n'

In [2]:
result

{'password': "b'$2b$12$q7jP7P5T2BTaZOaKgwF87e6exDCZW2ZV5JFNJfTR9u8RpqNFmGgLe'"}

In [3]:
result['password']

"b'$2b$12$q7jP7P5T2BTaZOaKgwF87e6exDCZW2ZV5JFNJfTR9u8RpqNFmGgLe'"